In [1]:
import csv
import os
from collections import defaultdict
import pandas as pd
from nltk.corpus import stopwords
from textblob import TextBlob, Word
from gensim.scripts.glove2word2vec import glove2word2vec
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import KeyedVectors # load the Stanford GloVe model
import ftfy
import string
from nltk.tokenize import word_tokenize

#nltk.download()

C:\Users\Naini\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
#reading csv
train = pd.read_csv('output.csv', encoding='ISO-8859-1',low_memory=False)
#train

In [3]:
train = train[train.notnull()]
#train

In [4]:
train = train.dropna(how='any') 
train

,title,content
0,House Republicans Fret About Winning Their Hea...,WASHINGTON Ã¢ÂÂ Congressional Republicans...
1,Rift Between Officers and Residents as Killing...,"After the bullet shells get counted, the blood..."
2,"Tyrus Wong, Ã¢ÂÂBambiÃ¢ÂÂ Artist Thwarted ...",When Walt DisneyÃ¢ÂÂs Ã¢ÂÂBambiÃ¢ÂÂ open...
3,"Among Deaths in 2016, a Heavy Toll in Pop Musi...","Death may be the great equalizer, but it isnÃ¢..."
4,Kim Jong-un Says North Korea Is Preparing to T...,"SEOUL, South Korea Ã¢ÂÂ North KoreaÃ¢ÂÂ..."
5,"Sick With a Cold, Queen Elizabeth Misses New Y...","LONDON Ã¢ÂÂ Queen Elizabeth II, who has b..."
6,TaiwanÃ¢ÂÂs President Accuses China of Renew...,BEIJING Ã¢ÂÂ President Tsai of Taiwan s...
7,"After Ã¢ÂÂThe Biggest Loser,Ã¢ÂÂ Their Bod...","Danny Cahill stood, slightly dazed, in a blizz..."
8,"First, a Mixtape. Then a Romance. - The New Yo...","Just how is Hillary Kerr, the founder of ..."
9,Calling on Angels While Enduring the Trials of...,Angels are everywhere in the MuÃÂ±iz familyÃ¢...


In [5]:
heads = train['title']
heads

0         House Republicans Fret About Winning Their Hea...
1         Rift Between Officers and Residents as Killing...
2         Tyrus Wong, Ã¢ÂÂBambiÃ¢ÂÂ Artist Thwarted ...
3         Among Deaths in 2016, a Heavy Toll in Pop Musi...
4         Kim Jong-un Says North Korea Is Preparing to T...
5         Sick With a Cold, Queen Elizabeth Misses New Y...
6         TaiwanÃ¢ÂÂs President Accuses China of Renew...
7         After Ã¢ÂÂThe Biggest Loser,Ã¢ÂÂ Their Bod...
8         First, a Mixtape. Then a Romance. - The New Yo...
9         Calling on Angels While Enduring the Trials of...
10        Weak Federal Powers Could Limit TrumpÃ¢ÂÂs C...
11        Can Carbon Capture Technology Prosper Under Tr...
12        Mar-a-Lago, the Future Winter White House and ...
13        How to form healthy habits in your 20s - The N...
14        Turning Your Vacation Photos Into Works of Art...
15        As Second Avenue Subway Opens, a Train Delay E...
16        Dylann Roof Himself Rejects Be

In [6]:
descs = train['content']
descs

0         WASHINGTON  Ã¢ÂÂ   Congressional Republicans...
1         After the bullet shells get counted, the blood...
2         When Walt DisneyÃ¢ÂÂs Ã¢ÂÂBambiÃ¢ÂÂ open...
3         Death may be the great equalizer, but it isnÃ¢...
4         SEOUL, South Korea  Ã¢ÂÂ   North KoreaÃ¢ÂÂ...
5         LONDON  Ã¢ÂÂ   Queen Elizabeth II, who has b...
6         BEIJING  Ã¢ÂÂ   President Tsai   of Taiwan s...
7         Danny Cahill stood, slightly dazed, in a blizz...
8         Just how   is Hillary Kerr, the    founder of ...
9         Angels are everywhere in the MuÃÂ±iz familyÃ¢...
10        With Donald J. Trump about to take control of ...
11        THOMPSONS, Tex.  Ã¢ÂÂ   Can one of the most ...
12        WEST PALM BEACH, Fla.  Ã¢ÂÂ   When   Donald ...
13        This article is part of a series aimed at help...
14        ItÃ¢ÂÂs the season for family travel and pho...
15        Finally. The Second Avenue subway opened in Ne...
16          pages into the   journal fou

In [7]:
heads = heads[:50]
descs = descs[:50]

In [8]:
title_list = []
for i in heads:
    title = ftfy.fix_text(i)
    title_list.append(title)
    #print(title)
    #print('---------------')    

In [9]:
title_list

['House Republicans Fret About Winning Their Health Care Suit - The New York Times',
 'Rift Between Officers and Residents as Killings Persist in South Bronx - The New York Times',
 "Tyrus Wong, 'Bambi' Artist Thwarted by Racial Bias, Dies at 106 - The New York Times",
 'Among Deaths in 2016, a Heavy Toll in Pop Music - The New York Times',
 'Kim Jong-un Says North Korea Is Preparing to Test Long-Range Missile - The New York Times',
 "Sick With a Cold, Queen Elizabeth Misses New Year's Service - The New York Times",
 "Taiwan's President Accuses China of Renewed Intimidation - The New York Times",
 "After 'The Biggest Loser,' Their Bodies Fought to Regain Weight - The New York Times",
 'First, a Mixtape. Then a Romance. - The New York Times',
 'Calling on Angels While Enduring the Trials of Job - The New York Times',
 "Weak Federal Powers Could Limit Trump's Climate-Policy Rollback - The New York Times",
 'Can Carbon Capture Technology Prosper Under Trump? - The New York Times',
 'Mar-a

In [10]:
content_list = []
for i in descs:
    descs = ftfy.fix_text(i)
    content_list.append(descs)
    #print(descs)
    #print('---------------')    

In [11]:
content_list

['WASHINGTON  —   Congressional Republicans have a new fear when it comes to their    health care lawsuit against the Obama administration: They might win. The incoming Trump administration could choose to no longer defend the executive branch against the suit, which challenges the administration\'s authority to spend billions of dollars on health insurance subsidies for   and   Americans, handing House Republicans a big victory on    issues. But a sudden loss of the disputed subsidies could conceivably cause the health care program to implode, leaving millions of people without access to health insurance before Republicans have prepared a replacement. That could lead to chaos in the insurance market and spur a political backlash just as Republicans gain full control of the government. To stave off that outcome, Republicans could find themselves in the awkward position of appropriating huge sums to temporarily prop up the Obama health care law, angering conservative voters who have bee

In [40]:
data = pd.read_pickle('full_data.pickle')
data

,descs,heads
0,WASHINGTON — Congressional Republicans have...,House Republicans Fret About Winning Their Hea...
1,"After the bullet shells get counted, the blood...",Rift Between Officers and Residents as Killing...
2,"When Walt Disney's ""Bambi"" opened in 1942, cri...","Tyrus Wong, 'Bambi' Artist Thwarted by Racial ..."
3,"Death may be the great equalizer, but it isn't...","Among Deaths in 2016, a Heavy Toll in Pop Musi..."
4,"SEOUL, South Korea — North Korea's leader, ...",Kim Jong-un Says North Korea Is Preparing to T...
5,"LONDON — Queen Elizabeth II, who has been b...","Sick With a Cold, Queen Elizabeth Misses New Y..."
6,BEIJING — President Tsai of Taiwan sharpl...,Taiwan's President Accuses China of Renewed In...
7,"Danny Cahill stood, slightly dazed, in a blizz...","After 'The Biggest Loser,' Their Bodies Fought..."
8,"Just how is Hillary Kerr, the founder of ...","First, a Mixtape. Then a Romance. - The New Yo..."
9,Angels are everywhere in the Muñiz family's ap...,Calling on Angels While Enduring the Trials of...


In [41]:
title = data['heads']
content = data['descs']

In [43]:
content

0         WASHINGTON  —   Congressional Republicans have...
1         After the bullet shells get counted, the blood...
2         When Walt Disney's "Bambi" opened in 1942, cri...
3         Death may be the great equalizer, but it isn't...
4         SEOUL, South Korea  —   North Korea's leader, ...
5         LONDON  —   Queen Elizabeth II, who has been b...
6         BEIJING  —   President Tsai   of Taiwan sharpl...
7         Danny Cahill stood, slightly dazed, in a blizz...
8         Just how   is Hillary Kerr, the    founder of ...
9         Angels are everywhere in the Muñiz family's ap...
10        With Donald J. Trump about to take control of ...
11        THOMPSONS, Tex.  —   Can one of the most promi...
12        WEST PALM BEACH, Fla.  —   When   Donald J. Tr...
13        This article is part of a series aimed at help...
14        It's the season for family travel and photos  ...
15        Finally. The Second Avenue subway opened in Ne...
16          pages into the   journal fou

In [42]:
title

0         House Republicans Fret About Winning Their Hea...
1         Rift Between Officers and Residents as Killing...
2         Tyrus Wong, 'Bambi' Artist Thwarted by Racial ...
3         Among Deaths in 2016, a Heavy Toll in Pop Musi...
4         Kim Jong-un Says North Korea Is Preparing to T...
5         Sick With a Cold, Queen Elizabeth Misses New Y...
6         Taiwan's President Accuses China of Renewed In...
7         After 'The Biggest Loser,' Their Bodies Fought...
8         First, a Mixtape. Then a Romance. - The New Yo...
9         Calling on Angels While Enduring the Trials of...
10        Weak Federal Powers Could Limit Trump's Climat...
11        Can Carbon Capture Technology Prosper Under Tr...
12        Mar-a-Lago, the Future Winter White House and ...
13        How to form healthy habits in your 20s - The N...
14        Turning Your Vacation Photos Into Works of Art...
15        As Second Avenue Subway Opens, a Train Delay E...
16        Dylann Roof Himself Rejects Be

In [44]:

title_list = [''.join(c for c in s if c not in string.punctuation) for s in title]


In [45]:
title_list

['House Republicans Fret About Winning Their Health Care Suit  The New York Times',
 'Rift Between Officers and Residents as Killings Persist in South Bronx  The New York Times',
 'Tyrus Wong Bambi Artist Thwarted by Racial Bias Dies at 106  The New York Times',
 'Among Deaths in 2016 a Heavy Toll in Pop Music  The New York Times',
 'Kim Jongun Says North Korea Is Preparing to Test LongRange Missile  The New York Times',
 'Sick With a Cold Queen Elizabeth Misses New Years Service  The New York Times',
 'Taiwans President Accuses China of Renewed Intimidation  The New York Times',
 'After The Biggest Loser Their Bodies Fought to Regain Weight  The New York Times',
 'First a Mixtape Then a Romance  The New York Times',
 'Calling on Angels While Enduring the Trials of Job  The New York Times',
 'Weak Federal Powers Could Limit Trumps ClimatePolicy Rollback  The New York Times',
 'Can Carbon Capture Technology Prosper Under Trump  The New York Times',
 'MaraLago the Future Winter White Hou

In [46]:
content_list = [''.join(c for c in s if c not in string.punctuation) for s in content]


In [47]:
content_list

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [48]:
tokenized_title = [word_tokenize(i) for i in title_list]

In [49]:
tokenized_title

[['House',
  'Republicans',
  'Fret',
  'About',
  'Winning',
  'Their',
  'Health',
  'Care',
  'Suit',
  'The',
  'New',
  'York',
  'Times'],
 ['Rift',
  'Between',
  'Officers',
  'and',
  'Residents',
  'as',
  'Killings',
  'Persist',
  'in',
  'South',
  'Bronx',
  'The',
  'New',
  'York',
  'Times'],
 ['Tyrus',
  'Wong',
  'Bambi',
  'Artist',
  'Thwarted',
  'by',
  'Racial',
  'Bias',
  'Dies',
  'at',
  '106',
  'The',
  'New',
  'York',
  'Times'],
 ['Among',
  'Deaths',
  'in',
  '2016',
  'a',
  'Heavy',
  'Toll',
  'in',
  'Pop',
  'Music',
  'The',
  'New',
  'York',
  'Times'],
 ['Kim',
  'Jongun',
  'Says',
  'North',
  'Korea',
  'Is',
  'Preparing',
  'to',
  'Test',
  'LongRange',
  'Missile',
  'The',
  'New',
  'York',
  'Times'],
 ['Sick',
  'With',
  'a',
  'Cold',
  'Queen',
  'Elizabeth',
  'Misses',
  'New',
  'Years',
  'Service',
  'The',
  'New',
  'York',
  'Times'],
 ['Taiwans',
  'President',
  'Accuses',
  'China',
  'of',
  'Renewed',
  'Intimidatio

In [50]:
tokenized_content = [word_tokenize(i) for i in content_list]

In [51]:
tokenized_content

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [52]:
stop = stopwords.words('english')

In [53]:
filtered_title = [word for word in tokenized_title if word not in stop]


In [54]:
filtered_title

[['House',
  'Republicans',
  'Fret',
  'About',
  'Winning',
  'Their',
  'Health',
  'Care',
  'Suit',
  'The',
  'New',
  'York',
  'Times'],
 ['Rift',
  'Between',
  'Officers',
  'and',
  'Residents',
  'as',
  'Killings',
  'Persist',
  'in',
  'South',
  'Bronx',
  'The',
  'New',
  'York',
  'Times'],
 ['Tyrus',
  'Wong',
  'Bambi',
  'Artist',
  'Thwarted',
  'by',
  'Racial',
  'Bias',
  'Dies',
  'at',
  '106',
  'The',
  'New',
  'York',
  'Times'],
 ['Among',
  'Deaths',
  'in',
  '2016',
  'a',
  'Heavy',
  'Toll',
  'in',
  'Pop',
  'Music',
  'The',
  'New',
  'York',
  'Times'],
 ['Kim',
  'Jongun',
  'Says',
  'North',
  'Korea',
  'Is',
  'Preparing',
  'to',
  'Test',
  'LongRange',
  'Missile',
  'The',
  'New',
  'York',
  'Times'],
 ['Sick',
  'With',
  'a',
  'Cold',
  'Queen',
  'Elizabeth',
  'Misses',
  'New',
  'Years',
  'Service',
  'The',
  'New',
  'York',
  'Times'],
 ['Taiwans',
  'President',
  'Accuses',
  'China',
  'of',
  'Renewed',
  'Intimidatio

In [55]:
filtered_content = [word for word in tokenized_content if word not in stop]


In [56]:
filtered_content

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [57]:
title_new = [' '.join(c for c in s if c not in string.punctuation) for s in filtered_title]

In [58]:
content_new = [' '.join(c for c in s if c not in string.punctuation) for s in filtered_content]

In [59]:
title_new

['House Republicans Fret About Winning Their Health Care Suit The New York Times',
 'Rift Between Officers and Residents as Killings Persist in South Bronx The New York Times',
 'Tyrus Wong Bambi Artist Thwarted by Racial Bias Dies at 106 The New York Times',
 'Among Deaths in 2016 a Heavy Toll in Pop Music The New York Times',
 'Kim Jongun Says North Korea Is Preparing to Test LongRange Missile The New York Times',
 'Sick With a Cold Queen Elizabeth Misses New Years Service The New York Times',
 'Taiwans President Accuses China of Renewed Intimidation The New York Times',
 'After The Biggest Loser Their Bodies Fought to Regain Weight The New York Times',
 'First a Mixtape Then a Romance The New York Times',
 'Calling on Angels While Enduring the Trials of Job The New York Times',
 'Weak Federal Powers Could Limit Trumps ClimatePolicy Rollback The New York Times',
 'Can Carbon Capture Technology Prosper Under Trump The New York Times',
 'MaraLago the Future Winter White House and Home 

In [60]:
content_new

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [61]:
final_list = pd.DataFrame(
    {'heads': title_new,
     'descs': content_new,
    })

In [62]:
final_list.to_pickle('tokenized_data.pickle')

In [63]:
df2 = pd.read_pickle('tokenized_data.pickle')
df2

,descs,heads
0,WASHINGTON — Congressional Republicans have a ...,House Republicans Fret About Winning Their Hea...
1,After the bullet shells get counted the blood ...,Rift Between Officers and Residents as Killing...
2,When Walt Disneys Bambi opened in 1942 critics...,Tyrus Wong Bambi Artist Thwarted by Racial Bia...
3,Death may be the great equalizer but it isnt n...,Among Deaths in 2016 a Heavy Toll in Pop Music...
4,SEOUL South Korea — North Koreas leader Kim sa...,Kim Jongun Says North Korea Is Preparing to Te...
5,LONDON — Queen Elizabeth II who has been battl...,Sick With a Cold Queen Elizabeth Misses New Ye...
6,BEIJING — President Tsai of Taiwan sharply cri...,Taiwans President Accuses China of Renewed Int...
7,Danny Cahill stood slightly dazed in a blizzar...,After The Biggest Loser Their Bodies Fought to...
8,Just how is Hillary Kerr the founder of a digi...,First a Mixtape Then a Romance The New York Times
9,Angels are everywhere in the Muñiz familys apa...,Calling on Angels While Enduring the Trials of...


In [71]:
import pickle
favorite_color = pickle.load( open( "tokenized_data.pickle", "rb" ) )

In [72]:
favorite_color

,descs,heads
0,WASHINGTON — Congressional Republicans have a ...,House Republicans Fret About Winning Their Hea...
1,After the bullet shells get counted the blood ...,Rift Between Officers and Residents as Killing...
2,When Walt Disneys Bambi opened in 1942 critics...,Tyrus Wong Bambi Artist Thwarted by Racial Bia...
3,Death may be the great equalizer but it isnt n...,Among Deaths in 2016 a Heavy Toll in Pop Music...
4,SEOUL South Korea — North Koreas leader Kim sa...,Kim Jongun Says North Korea Is Preparing to Te...
5,LONDON — Queen Elizabeth II who has been battl...,Sick With a Cold Queen Elizabeth Misses New Ye...
6,BEIJING — President Tsai of Taiwan sharply cri...,Taiwans President Accuses China of Renewed Int...
7,Danny Cahill stood slightly dazed in a blizzar...,After The Biggest Loser Their Bodies Fought to...
8,Just how is Hillary Kerr the founder of a digi...,First a Mixtape Then a Romance The New York Times
9,Angels are everywhere in the Muñiz familys apa...,Calling on Angels While Enduring the Trials of...
